In [1]:
# load autoreloader
%reload_ext autoreload
%autoreload 2

In [31]:
# import memory_maze
import gym, os

# import tensorflow as tf
# os.environ["DISPLAY"] = ":0"
# os.environ['MUJOCO_GL'] = 'glfw'
# os.environ['PYOPENGL_PLATFORM'] = 'glfw'
# PYOPENGL_PLATFORM=egl
# from tf_agents.environments import suite_gym, tf_py_environment
import numpy as np
import torch
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import (
    BaseCallback,
    CallbackList,
    CheckpointCallback,
    EvalCallback,
)
from stable_baselines3.common.monitor import Monitor

from sb3 import model_vis_vec as model_lib

In [17]:
# os.environ['MUJOCO_GL']

In [18]:
# !export MUJOCO_GL="glfw"
# %env MUJOCO_GL=egl

env = gym.make('memory_maze:MemoryMaze-9x9-v0')
obs = env.reset()

In [19]:
obs

array([[[ 80, 137,  71],
        [ 80, 137,  71],
        [ 80, 137,  71],
        ...,
        [ 80, 137,  71],
        [ 80, 137,  71],
        [ 80, 137,  71]],

       [[ 80, 137,  71],
        [ 80, 137,  71],
        [ 80, 137,  71],
        ...,
        [ 80, 137,  71],
        [ 80, 137,  71],
        [ 80, 137,  71]],

       [[ 80, 137,  71],
        [ 80, 137,  71],
        [ 22,  78, 118],
        ...,
        [ 32, 117, 179],
        [ 80, 137,  71],
        [ 80, 137,  71]],

       ...,

       [[ 80, 137,  71],
        [ 80, 137,  71],
        [ 32, 117, 178],
        ...,
        [ 31, 111, 169],
        [ 80, 137,  71],
        [ 80, 137,  71]],

       [[ 80, 137,  71],
        [ 80, 137,  71],
        [ 80, 137,  71],
        ...,
        [ 80, 137,  71],
        [ 80, 137,  71],
        [ 80, 137,  71]],

       [[ 80, 137,  71],
        [ 80, 137,  71],
        [ 80, 137,  71],
        ...,
        [ 80, 137,  71],
        [ 80, 137,  71],
        [ 80, 137,  71]]

In [7]:
import stable_baselines3

In [5]:
stable_baselines3.common.env_checker.check_env

AttributeError: module 'stable_baselines3.common' has no attribute 'env_checker'

In [20]:
import memory_maze

In [21]:
env = memory_maze.tasks.memory_maze_9x9()

In [22]:
env

In [13]:
!pip install shimmy[dm-control]

  Using cached dm_control-1.0.15-py3-none-any.whl.metadata (1.3 kB)
  Using cached mujoco-3.0.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (44 kB)
  Using cached pyparsing-3.1.1-py3-none-any.whl.metadata (5.1 kB)
Using cached dm_control-1.0.15-py3-none-any.whl (55.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 13.1 MB/s eta 0:00:00m eta 0:00:010:01
Using cached mujoco-3.0.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.3 MB)
Using cached pyparsing-3.1.1-py3-none-any.whl (103 kB)
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 2.4.7
    Uninstalling pyparsing-2.4.7:
      Successfully uninstalled pyparsing-2.4.7
  Attempting uninstall: mujoco
    Found existing installation: mujoco 2.1.5
    Uninstalling mujoco-2.1.5:
      Successfully uninstalled mujoco-2.1.5
  You can safely remove it manually.
  Attempting uninstall: dm-control
    Found existing installation: dm-control 1.0.2
    Uninstalling dm-control

In [24]:
from shimmy.dm_control_compatibility import DmControlCompatibilityV0
import dm_control.composer as composer
import dm_control.rl.control as control
import dm_env

# Example: Wrapping a dm_control environment
# dm_control_env = control.Environment()  # or composer.Environment()
env = DmControlCompatibilityV0(env=env, render_mode=None)

In [25]:
env

In [32]:
class EpisodicRewardCallback(BaseCallback):
    def __init__(self, verbose=0, plot_freq=10, log_dir="./tensorboard_logs/"):
        super(EpisodicRewardCallback, self).__init__(verbose)
        self.episode_rewards = []
        self.plot_freq = plot_freq
        self.log_dir = log_dir
        self.ep_rewards = 0.0

    def on_rollout_start(self) -> None:
        # print("starting rollout")
        self.ep_rewards = 0.0

    def _on_step(self):
        self.ep_rewards += self.locals["rewards"][-1]

    def _on_rollout_end(self) -> None:
        # print("ending rollout")
        episode_reward = self.ep_rewards
        self.episode_rewards.append(episode_reward)
        # print(f"Episode reward: {episode_reward}")
        with open("rewards.npy", "wb") as f:
            np.save(file=f, arr=np.array(self.episode_rewards))

    # def _on_training_end(self) -> None:
    #     self._plot_rewards()

    # def _plot_rewards(self):
    #     x = np.arange(1, len(self.episode_rewards) + 1)
    #     plt.figure(figsize=(10, 5))
    #     plt.plot(x, self.episode_rewards)
    #     plt.xlabel('Episode')
    #     plt.ylabel('Total Reward')
    #     plt.title('Episodic Rewards')
    #     plt.grid(True)
    #     plt.savefig(os.path.join(self.log_dir, 'episodic_rewards.png'))
    #     plt.show()


def linear_schedule(initial_lr, final_lr, total_steps):
    def schedule(progress):
        fraction = min(progress / total_steps, 1.0)
        return initial_lr + fraction * (final_lr - initial_lr)

    return schedule


def custom_entropy_schedule(initial_entropy_coef, final_entropy_coef, total_steps):
    def entropy_coef_schedule(progress):
        fraction = min(progress / total_steps, 1.0)
        return torch.tensor(
            initial_entropy_coef
            + fraction * (final_entropy_coef - initial_entropy_coef),
            dtype=torch.float32,
        )

    return entropy_coef_schedule


TOTAL_TIMESTEPS = 170_000_000
policy = model_lib.CustomPolicy(
    observation_space=env.observation_space,
    action_space=env.action_space,
    lr_schedule=linear_schedule(
        initial_lr=3e-4, final_lr=1e-4, total_steps=TOTAL_TIMESTEPS
    ),
)  # custom policy network

In [29]:
observation_space

NameError: name 'observation_space' is not defined

In [30]:
env.observation_space

Dict('image': Box(0, 255, (64, 64, 3), uint8), 'target_color': Box(-inf, inf, (3,), float64))